# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [1]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import yaml

# Token usage tracking
from token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage_with_estimate,
    log_token_usage,
)

from tools import (
    create_chat_completion_client,
    current_datetime_utc_tool,
    adjust_properties_for_fixed_response,
    get_tavily_search_tool,
    get_attributes,
)

#### Note: You need a tavily API key to use the web search tool!

In [ ]:
# load model info from resources.yaml
with open("resources.yaml", "r") as f:
    resources = yaml.load(f, Loader=yaml.SafeLoader)

# Check if web search is possible (i.e., if API key is set)
ALLOW_WEB_SEARCH = os.getenv("TAVILY_API_KEY", False)

In [3]:
# Prepare tools
basic_tools = {current_datetime_utc_tool}
web_search_tools = set()
if ALLOW_WEB_SEARCH:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = { await get_tavily_search_tool() }

Set the research topic
----------------------

In [4]:
# topic = "Impacts of Model Context Protocol on Agentic AI"
# topic = "What is Fraunhofer MEVIS?"
topic = "What day and time is it in Berlin, Germany?"
# topic = "What is the wheather like in Berlin, Germany currently?"
# topic = "What has Fraunhofer MEVIS done in the context of thoracic imaging?"

Select Model and Agent Configuration
-----------------------------

In [5]:
# Use a GPT-4.1-Nano for cost control during testing, GPT-5.1-Nano for better but slower results, GPT-5-Mini for best results
MODEL_NAME = "GPT-4.1-Nano"

model = [
    m for m in resources["models"] if m.get("name", "").lower() == MODEL_NAME.lower()
][0]
print("Using model:", model["name"], "type:", model["type"])

agentConfig = {
    "Researcher": dict(system_message="Gather and summarize factual info, add important web references.", tools=web_search_tools), 
    "FactChecker": dict(system_message="Use web search to verify facts and citations from the Researcher's response. For each found error, give a correct version and the web reference for verification.", tools=web_search_tools), 
    "Revisor": dict(system_message="Use the fact checker's results to revise the Researcher's report. Only if there were no fact issues, just repeat the Researcher's response. In both cases, add the fact checker's citations."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result to generate an improved response to the initial user question."),# Include the most important references."),
    "Editor": dict(system_message="Do some language polishing on the previous result to generate a final response to the initial question. Signal APPROVED when done."),
}

# For testing, we can fix an agent's response to a known answer
# adjust_properties_for_fixed_response(agentConfig["Researcher"], "Fraunhofer MEVIS is a research institute in Frankfurt for biological weapons. It was founded in 1995. See https://www.mefis.fraunhofer.de/en.html for more information.")

Using model: gpt-4.1-nano type: openai


In [ ]:
# Now create model client with token tracking
model_client = wrap_chat_client_for_tokens(create_chat_completion_client(model))

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    p = props.copy()
    p.setdefault("tools", set()).update(basic_tools)
    p.setdefault("reflect_on_tool_use", True)
    agents.append(
        AssistantAgent(
            name=agent_type,
            **p,
            model_client=model_client,
        )
    )

In [7]:
# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [8]:
# Create the host agent with deep dive capabilities and the host agent
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

Start the Deep Dive
===================

In [9]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await host.run(task=f"Deep dive on: {topic}")
    # Try native usage first; fallback to estimation
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 1448, completion: 1613, total: 3061 (method=tiktoken)
Approximate cost for this run: $0.0004 USD (input: $0.0001, output: $0.0003)
🔍 DeepDive result (took 9.98 sec.) resulted in 23 messages.


Explore Results
===============

In [10]:
# Display token usage after the deep dive
print("=== Token Usage Summary ===")
print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), model_info=model)

=== Token Usage Summary ===
Estimated token usage -> prompt: 1448, completion: 1613, total: 3061 (method=tiktoken)
Approximate cost for this run: $0.0004 USD (input: $0.0001, output: $0.0003)


In [ ]:
# Display the full result structure
print("Result type:", type(result))

print("\nResult attributes:")
print(get_attributes(result))
print("\nMessage attributes:")
print(get_attributes(result.messages[-1]))

for i, msg in enumerate(result.messages):
    tool_info = f"Tools: {[c.name for c in msg.content if hasattr(c, 'name')]}" if "tool" in msg.type.lower() else "no tool involved"
    if hasattr(msg, 'tool_calls'):
        tool_info += f", tool_calls: {getattr(msg, 'tool_calls', None)}"
    ts = msg.created_at.strftime("%H:%M:%S.%f")[:-3] if getattr(msg, "created_at", None) else "(unknown)"
    #print("")
    print(f"{i}: {msg.type} by {msg.source} at {ts}, {tool_info}")
    #print(msg.content)

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']

Message attributes:
['construct', 'content', 'copy', 'created_at', 'dict', 'dump', 'from_orm', 'id', 'json', 'load', 'metadata', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_st

In [12]:
# Clean up resources
await model_client.close()

In [13]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: None
Message Sequence:
 - Researcher
 - FactChecker
 - Revisor
 - Critic
 - Summarizer
 - Editor

 Researcher: The current Coordinated Universal Time (UTC) is approximately 14:57 on September 17, 2025. 

Berlin, Germany, is in the Central European Time Zone (CET), which is usually UTC+1 during standard time and UTC+2 during daylight saving time. Since it is September, daylight saving time is still in effect, so Berlin is on Central European Summer Time (CEST), which is UTC+2.

Therefore, the current local date and time in Berlin, Germany, is approximately **16:57 on September 17, 2025**.

For real-time updates, you can also visit [Time and Date - Berlin](https://www.timeanddate.com/worldclock/germany/berlin).

FactChecker: The current Coordinated Universal Time (UTC) is approximately 14:57 on September 17, 2025. Since Berlin is in the Central European Time Zone (CET) and observes daylight saving time (CEST) during September, it is UTC+2.

Therefore, the current local time 

In [14]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-17T14:57:35.280870+00:00', 'topic': 'What day and time is it in Berlin, Germany?', 'prompt': 1448, 'completion': 1613, 'total': 3061, 'method': 'tiktoken', 'cost_usd': 0.000395, 'input_cost_usd': 7.2e-05, 'output_cost_usd': 0.000323}


{'timestamp': '2025-09-17T14:57:35.280870+00:00',
 'topic': 'What day and time is it in Berlin, Germany?',
 'prompt': 1448,
 'completion': 1613,
 'total': 3061,
 'method': 'tiktoken',
 'cost_usd': 0.000395,
 'input_cost_usd': 7.2e-05,
 'output_cost_usd': 0.000323}